# T4 Form Extraction

In [94]:
import fitz
import numpy as np
import cv2 as cv

import os
import pytesseract
import cv2
import pandas as pd
import time

import re
import os
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from utils import *
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from detectron2.engine import DefaultPredictor


cfg_save_path = "t4a_cfg.pickle"

with open(cfg_save_path, 'rb') as f:
    cfg = pickle.load(f)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8

predictor = DefaultPredictor(cfg)


class PDF_T4a_ImageExtractor:
    """
    page_number=1 (Page Number to convert into image)
    """
    def __init__(self, pdf_file_path, output_image_path,template_json,page_number=1):
        self.pdf_file_path = pdf_file_path
        self.output_image_path = output_image_path
        self.page_number = page_number
        self.template_json = template_json
        self.mapping_dict = {0: "Social Insurance Number",
                             1: "receipents program account number",
                             2: "Pension or Superannuation-line 11500",
                             3: "lum-sum amount-line 13000",
                             4: "self-employed commisions",
                             5: "income-tax deducted-line 43700",
                             6: "Annuities renties",
                             7: "fees for services",
                             8: "payers program account number",
                             9: "box_case amount montant_1",
                             10: "box_case amount montant_10",
                             11: "box_case amount montant_11",
                             12: "box_case amount montant_12",
                             13: "box_case amount montant_2",
                             14: "box_case amount montant_3",
                             15: "box_case amount montant_4",
                             16: "box_case amount montant_5",
                             17: "box_case amount montant_6",
                             18: "box_case amount montant_7",
                             19: "box_case amount montant_8",
                             20: "box_case amount montant_9",
                             21: "payersname",
                             22: "recipients name and address",
                             23: "Year"}
        

    def convert_pdf_to_image(self):
        try:
            pdf_document = fitz.open(self.pdf_file_path)
            dpi = 300
            first_page = pdf_document.load_page(self.page_number - 1)
            pix = first_page.get_pixmap(matrix=fitz.Matrix(dpi / 72, dpi / 72))
            pix.save(self.output_image_path)
            
            # Load the image
            image = cv2.imread(self.output_image_path)
            
            # Specify the upper and lower line coordinates (y-coordinates)
            upper_line_y = 62  # Adjust this value as needed
            lower_line_y = 1400  # Adjust this value as needed
            

            # Crop the image using the specified lines
            cropped_image = image[upper_line_y:lower_line_y, :]
            cv2.imwrite(self.output_image_path, cropped_image)
            
            pdf_document.close()
            print(f'Conversion of the first page to {self.output_image_path} completed.')
            return True
        except Exception as e:
            print(f"Error: Pdf to Image Conversion failed!! {str(e)}")
            return False
        
    # Function to remove fuzzy matched substring
    def remove_fuzzy_substring(self,original_str, substring):
        ratio = fuzz.token_set_ratio(original_str, substring)
        if ratio > 50:
            return original_str.replace(substring, '')
        return original_str

    def final_json_creation(self,final_mapping):
        final_mapping_modified = {}
        for key, value in final_mapping.items():
            if key in ['payersname']:
                # Keep the original value for specified keys
                final_mapping_modified[key] = value
            elif key in ['recipients name and address']:
                final_mapping_modified[key] = value_cleaned[125:].strip()
            else:
                # Remove non-integer characters, excluding "/", and limit consecutive spaces to three
                value_cleaned = re.sub(r'(?<!/)\s{3,}', '', ''.join(char if char.isdigit() or char == '/' or char.isspace() else '' for char in value))
                if key in ['payers program account number']:
                    final_mapping_modified[key] = value_cleaned[3:]
                else:
                    final_mapping_modified[key] = value_cleaned
            return final_mapping_modified
        
    def process_pdf_to_text(self):
        if self.convert_pdf_to_image():
            cropped_images_dict = crop_image(image_path, predictor)
            # Create a new dictionary by mapping values from mapping_dict to keys of text_dict
            final_mapping = {self.mapping_dict[key]: value for key, value in cropped_images_dict.items()}
            final_mapping_modified = final_json_creation(final_mapping)

            print("True",final_mapping_modified)

ModuleNotFoundError: No module named 'detectron2.data'

In [75]:
template_json = {
    'recipients name and address': '',
    'Annuities renties': '',
    'payers program account number': '',
    'Year': '',
    'box_case amount montant_8': '',
    'box_case amount montant_2': '',
    'payersname': '',
    'box_case amount montant_9': '',
    'box_case amount montant_4': '',
    'box_case amount montant_12': '',
    'self-employed commisions': '',
    'lum-sum amount-line 13000': '',
    'fees for services': '',
    'box_case amount montant_1': '',
    'box_case amount montant_11': '',
    'box_case amount montant_6': '',
    'box_case amount montant_7': '',
    'Pension or Superannuation-line 11500': '',
    'receipents program account number': '',
    'box_case amount montant_10': '',
    'income-tax deducted-line 43700': '',
    'box_case amount montant_5': '',
    'box_case amount montant_3': '',
    'Social Insurance Number': ''
}


pdf_file_path = "T4a-Scanned-23e.pdf"
output_image_path = "T4a-Scanned-23e.png"

pdf_extractor = PDF_T4a_ImageExtractor(pdf_file_path, output_image_path,template_json)
output_json = pdf_extractor.process_pdf_to_text(pdf_file_path)

Conversion of the first page to T4a-Scanned-23e.png completed.
True


In [88]:
len("Recipient's name and address Nom et adresse du beneficiaire Last name (print) Nom de famille (en lettres moulees) First name")

124

-------------------------------

# T4ps Form Extraction

In [1]:
import fitz
import numpy as np
import cv2 as cv

import os
import pytesseract
import cv2
import time
import pickle
import re
import os
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from utils import *
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from detectron2.engine import DefaultPredictor


cfg_save_path = "t4ps_cfg.pickle"

with open(cfg_save_path, 'rb') as f:
    cfg = pickle.load(f)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)


class PDF_T4ps_ImageExtractor:
    """
    page_number=1 (Page Number to convert into image)
    """
    def __init__(self, pdf_file_path, output_image_path,template_json,page_number=1):
        self.pdf_file_path = pdf_file_path
        self.output_image_path = output_image_path
        self.page_number = page_number
        self.template_json = template_json
        self.mapping_dict = {0: "Actual amount of dividends other than eligible dividends",
                             1: "Taxable amount of dividends other than eligible dividends",
                             2: "Dividend tax credit for dividends other than eligible dividends",
                             3: "Actual amount of eligible dividends",
                             4: "Taxable amount of eligible dividends",
                             5: "Dividend tax credit eligible dividends",
                             6: "Capital gain or losses",
                             7: "Other employement income",
                             8: "Total Amount forfeited due to withdrawal from plan",
                             9: "Foreign Non business income",
                             10: "Foreign Capital Gains or losses",
                             11: "Foreign Non business income tax",
                             12: "Specified Employee",
                             13: "Employees profit sharing plan contributions",
                             14: "Employer's account number",
                             15: "Employee's name",
                             16: "Employee's address",
                             17: "Employer's name",
                             18: "Name of employees profit sharing plan",
                             19: "Social insurance number",
                             20: "Year"}
        

    def convert_pdf_to_image(self):
        try:
            pdf_document = fitz.open(self.pdf_file_path)
            dpi = 300
            first_page = pdf_document.load_page(self.page_number - 1)
            pix = first_page.get_pixmap(matrix=fitz.Matrix(dpi / 72, dpi / 72))
            pix.save(self.output_image_path)
            
            # Load the image
            image = cv2.imread(self.output_image_path)
            
            # Specify the upper and lower line coordinates (y-coordinates)
            upper_line_y = 60  # Adjust this value as needed
            lower_line_y = 1600  # Adjust this value as needed
            

            # Crop the image using the specified lines
            cropped_image = image[upper_line_y:lower_line_y, :]
            cv2.imwrite(self.output_image_path, cropped_image)
            
            pdf_document.close()
            print(f'Conversion of the first page to {self.output_image_path} completed.')
            return True
        except Exception as e:
            print(f"Error: Pdf to Image Conversion failed!! {str(e)}")
            return False
    

    def final_json_creation(self,final_mapping):
        final_mapping_modified = {}
        for key, value in final_mapping.items():
            
            final_mapping_modified[key] = value
            return final_mapping_modified

    def crop_image(self,image_path, predictor):
        
        # reader = easyocr.Reader(['en'])
#         try:
#             reader = easyocr.Reader(['en'],  model_storage_directory='/pas-models/easy-ocr-temp', user_folder='/pas-models/easy-ocr-temp')
#         except:
#             try:
#                 reader = easyocr.Reader(['en'],  model_storage_directory='/pas-models/easy-ocr-temp')
#             except:
#                 reader = easyocr.Reader(['en'])
                
                
        reader = easyocr.Reader(['en'])
        im = cv2.imread(image_path)
        outputs = predictor(im)

        boxes_list = outputs["instances"].pred_boxes.tensor.cpu().numpy()
        scores_list = outputs["instances"].scores.detach().cpu().numpy()
        pred_classes_list = outputs["instances"].pred_classes.detach().cpu().numpy()

        cropped_images_dict = {}
        reader = easyocr.Reader(['en'])

        for i, (box, score, pred_cls) in enumerate(zip(boxes_list, scores_list, pred_classes_list), start=1):
            x_min, y_min, x_max, y_max = map(int, box)

            coord_dict = {}
            cropped_image = im[y_min:y_max, x_min:x_max, :]

            #Cordinates
            x = x_min
            y = y_min
            w = (x_max - x_min)
            h = (y_max - y_min)
            
            # Use EasyOCR to extract text
            results = reader.readtext(cropped_image)
            concatenated_text = ' '.join([item[1] for item in results])
        
            coord_dict['Value'] = concatenated_text.replace("/", ".")
            coord_dict['Coordinates'] = (y, y + h, x, x + w)
            cropped_images_dict[pred_cls] = coord_dict

            # Display the image using plt
            plt.figure(figsize=(8, 8))
            plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()
            print(concatenated_text)
            
        return cropped_images_dict
        
    def process_single_pdf_T4a(self,pdf_file_path):
        if self.convert_pdf_to_image():
            cropped_images_dict = self.crop_image(self.output_image_path, predictor)
            #Create a new dictionary by mapping values from mapping_dict to keys of text_dict
            final_mapping = {self.mapping_dict[key]: value for key, value in cropped_images_dict.items()}
            return final_mapping

ModuleNotFoundError: No module named 'detectron2.data'

In [ ]:
template_json = {'Actual amount of dividends other than eligible dividends': '', 
                 'Taxable amount of dividends other than eligible dividends': '',
                 'Dividend tax credit for dividends other than eligible dividends': '', 
                 'Actual amount of eligible dividends': '', 
                 'Taxable amount of eligible dividends': '',
                 'Dividend tax credit eligible dividends': '',
                 'Capital gain or losses': '',
                 'Other employement income': '',
                 'Total Amount forfeited due to withdrawal from plan': '',
                 'Foreign Non business income': '',
                 'Foreign Capital Gains or losses': '',
                 'Foreign Non business income tax': '',
                 'Specified Employee': '',
                 'Employees profit sharing plan contributions': '',
                 "Employer's account number": '',
                 "Employee's name": '',
                 "Employee's address": '',
                 "Employer's name": '',
                 'Name of employees profit sharing plan': '',
                 'Social insurance number': '',
                 'Year': ''}

In [ ]:
pdf_file_path = "t4ps-fill-23e scanned.pdf"
output_image_path = "t4ps-fill-23e scanned.png"

pdf_extractor = PDF_T4ps_ImageExtractor(pdf_file_path, output_image_path,template_json)
output_json = pdf_extractor.process_single_pdf_T4a(pdf_file_path)

In [2]:
output_json

NameError: name 'output_json' is not defined